In [ ]:
# import tensorflow as tf
# import numpy as np


In [ ]:
# def _bytes_feature(value):
#     """Returns a bytes_list from a string / byte."""
#     if isinstance(value, type(tf.constant(0))):
#         value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
#     return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def _float_feature(value):
#     """Returns a float_list from a float / double."""
#     return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# def _int64_feature(value):
#     """Returns an int64_list from a bool / enum / int / uint."""
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# def serialize_array(array):
#     array = tf.io.serialize_tensor(array)
#     return array


# def train_serialize_example(feature0,):
#     feature = {
#       'image'         : _bytes_feature(feature0),
#   }
#     example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
#     return example_proto.SerializeToString()



# with tf.io.TFRecordWriter('1.tfrec') as writer:
#     example  = train_serialize_example(
#                 serialize_array(np.array([5.6, 1.2, 3.6], dtype=np.float32))
#                 )

#     writer.write(example)

In [ ]:
# feature_description = {
#     'image': tf.io.FixedLenFeature([], tf.string),   
# }


# def _parse_function(example_proto):
#   # Parse the input `tf.Example` proto using the dictionary above.
#   return tf.io.parse_single_example(example_proto, feature_description)

In [ ]:
# ds = tf.data.TFRecordDataset(["1.tfrec"])
# ds = ds.map(_parse_function)

In [ ]:
# for a in ds.take(1):
# #     print(a)
#     print(tf.io.parse_tensor(a["image"], out_type=tf.float32).numpy())
#     break

In [ ]:
SEED  = 42
FOLDS = 15
# SAVE_FOLDS = list(range(FOLDS))


COMPRESS_UINT8 = False
if COMPRESS_UINT8:
    SAVE_FOLDS = list(range(FOLDS))
else:
#     SAVE_FOLDS = [0,/1,2,3,4,5]
    SAVE_FOLDS = []



SANITY_CHECK = False

In [ ]:
import numpy as np 
import pandas as pd 
import os, shutil
from glob import glob
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_label_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
test_label_df  = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
print("train df shape is ", train_label_df.shape)
print("test  df shape is ", test_label_df.shape)

In [ ]:
train_label_df.head()

In [ ]:
test_label_df.head()

In [ ]:
train_paths = glob('../input/seti-breakthrough-listen/train/**/*.npy')
test_paths = glob('../input/seti-breakthrough-listen/test/**/*.npy')
len(train_paths), len(test_paths)

In [ ]:
assert train_label_df.shape[0] == len(train_paths) and test_label_df.shape[0] == len(test_paths)

In [ ]:
train_df = pd.DataFrame({'filepath':train_paths})
train_df['id'] = train_df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
train_df['group'] = train_df.filepath.map(lambda x: x.split('/')[-2])
train_df = pd.merge(train_df, train_label_df, on='id', how='left')
train_df['group_target'] = train_df.group+train_df.target.astype(str)
train_df.head()

In [ ]:
test_df = pd.DataFrame({'filepath':test_paths})
test_df['id'] = test_df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
test_df['group'] = test_df.filepath.map(lambda x: x.split('/')[-2])
test_df.head()

# check data

In [ ]:
train_df.target.value_counts()

In [ ]:
train_df.group.value_counts()

In [ ]:
train_df.groupby(['group','target'])['id'].count()

In [ ]:
kfold_df = pd.read_csv("/kaggle/input/seti10daysinfo/train_kfold.csv")
train_df['fold'] = -1
train_df.set_index("id", inplace=True)
for i in range(kfold_df.shape[0]):
    sample = kfold_df.iloc[i]
    id = sample["id"]
    fold = sample["fold"]
    train_df.loc[id, "fold"] = fold
print(sorted(train_df.fold.unique()))
train_df.reset_index(inplace=True)
train_df.head()

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import GroupKFold
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED).split(train_df, train_df['target'])
# # skf = GroupKFold(n_splits=FOLDS).split(train_df, train_df['target'], train_df["group"])


# train_df['fold'] = -1
# for fold, (train_idx, val_idx) in enumerate(skf):
#     train_df.loc[val_idx,'fold'] = fold
# train_df.head()

In [ ]:
import matplotlib.pyplot as plt, cv2

def load_signal(filepath, dim=None, interpolation=None):
    sgnl = np.load(filepath)
    sgnl = sgnl[::2,] # we're taking only 1, 3, 5
    img  = np.moveaxis(sgnl, 0, -1)
    img  = img.astype(np.float32)
    if dim is not None:
        img = cv2.resize(img, dsize=(dim, dim), interpolation=cv2.INTER_NEAREST)
    return img

def visualize(sgnl):
    sgnl = sgnl.astype(float)
    plt.figure(figsize=(20, 10))
    for idx in range(3):
        plt.subplot(2, 6, idx+1)
        plt.imshow(sgnl[...,idx])
        plt.axis('OFF')
    plt.tight_layout()
    plt.show()
    
    
sgnl = load_signal(train_df[train_df.target==1].filepath.iloc[2])
visualize(sgnl)
sgnl.shape, sgnl.dtype

# check compress diff

In [ ]:
temp_fname = train_df[train_df.target==1].filepath.iloc[2]
temp_fname

In [ ]:
sgnl = load_signal(temp_fname)

sgnl_compress = cv2.imdecode(cv2.imencode(".png", sgnl)[1], cv2.IMREAD_COLOR)

sgnl_uint8 = sgnl.astype(np.uint8)
sgnl_compress_uint8 = cv2.imdecode(cv2.imencode(".png", sgnl_uint8)[1], cv2.IMREAD_COLOR)

print(np.sum(sgnl - sgnl_compress))
print(np.sum(sgnl - sgnl_uint8))
print(np.sum(sgnl - sgnl_compress_uint8))

In [ ]:
visualize(sgnl_compress)
visualize(sgnl_uint8)

# tfrecord

In [ ]:
import tensorflow as tf

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
    array = tf.io.serialize_tensor(array)
    return array

In [ ]:
def train_serialize_example(feature0, feature1, feature2, feature3):
    feature = {
      'image'         : _bytes_feature(feature0),
      'image_id'      : _bytes_feature(feature1),
      'group'         : _bytes_feature(feature2),    
      'target'        : _float_feature(feature3),
      }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# train

In [ ]:
x_tot,x2_tot = [],[]

In [ ]:
# folds = sorted(train_df.fold.unique().tolist())



folds = SAVE_FOLDS

for fold in tqdm(folds):
    fold_df = train_df[train_df.fold == fold]
    print('Writing TFRecord of fold %i :'%(fold))
    with tf.io.TFRecordWriter('train%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
        
        if SANITY_CHECK:
            image = load_signal(temp_fname, dim=None) # no resize
            if COMPRESS_UINT8:
                image = image.astype(np.uint8) # to uint8 for saving mem
#                 image = image.tobytes()
                image = cv2.imencode('.png', image)[1].tobytes()
            else:
                image = serialize_array(image)
                
            example  = train_serialize_example(
                image,
                b"aaaa",
                b"bbbbb",
                1.0,
                )

            writer.write(example)

        else:
            samples = fold_df.shape[0]
            it = tqdm(range(samples))
            for k in it:
                row = fold_df.iloc[k, :]
                image = load_signal(row["filepath"], dim=None) # no resize
                
                cc = image.shape[2]
                assert cc == 3 or cc == 6
                x_tot.append((image).reshape(-1,cc).mean(0))
                x2_tot.append(((image)**2).reshape(-1,cc).mean(0))
                
                if COMPRESS_UINT8:
                    image = image.astype(np.uint8) # to uint8 for saving mem
#                     image = image.tobytes()
                    image = cv2.imencode('.png', image)[1].tobytes()
                else:
                    image = serialize_array(image)

                image_id = row["id"]
                group = row["group"]
                target = np.array(row["target"], dtype=np.float32)

                example  = train_serialize_example(
                    image,
                    str.encode(image_id),
                    str.encode(group),
                    target,
                    )

                writer.write(example)
            
    if 1:
        filepath = 'train%.2i-%i.tfrec'%(fold,fold_df.shape[0])
        filename = filepath.split('/')[-1]
        filesize = os.path.getsize(filepath)/10**6
        print(filename,':',np.around(filesize, 2),'MB')  

# test

In [ ]:
def test_serialize_example(feature0, feature1, feature2):
    feature = {
      'image'         : _bytes_feature(feature0),
      'image_id'      : _bytes_feature(feature1),
      'group'         : _bytes_feature(feature2),    
  }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
if 1:
    folds = 10
    l     = int(np.ceil(test_df.shape[0]/folds))
    
    
    PROCESSING_FOLDS = [6,7,8,9]
    
    for fold in tqdm(PROCESSING_FOLDS): # create tfrecord for each fold
        fold_df = test_df.iloc[l*fold:l*(fold+1)]

        print('Writing TFRecord of fold %i :'%(fold))
        with tf.io.TFRecordWriter('test%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
            samples = fold_df.shape[0]
            it = tqdm(range(samples))
            for k in it:
                row = fold_df.iloc[k, :]
                image = load_signal(row["filepath"], dim=None) # no resize
                
                
                cc = image.shape[2]
                assert cc == 3 or cc == 6
                x_tot.append((image).reshape(-1,cc).mean(0))
                x2_tot.append(((image)**2).reshape(-1,cc).mean(0))
                
                
                if COMPRESS_UINT8:
                    image = image.astype(np.uint8) # to uint8 for saving mem
#                     image = image.tobytes()
                    image = cv2.imencode('.png', image)[1].tobytes()
                else:
                    image = serialize_array(image)

                image_id = row["id"]
                group = row["group"]

                example  = test_serialize_example(
                    image,
                    str.encode(image_id),
                    str.encode(group),
                    )

                writer.write(example)

        if 1:
            filepath = 'test%.2i-%i.tfrec'%(fold,fold_df.shape[0])
            filename = filepath.split('/')[-1]
            filesize = os.path.getsize(filepath)/10**6
            print(filename,':',np.around(filesize, 2),'MB')  


# sanity check

In [ ]:
import re, math


def decode_image(image_data):
#     image = tf.io.decode_raw(image_data, tf.uint8)
#     image = tf.image.decode_png(image_data, channels=3)
    image = tf.io.parse_tensor(image_data, tf.float32)
    image = tf.cast(image, tf.float32)
#     image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [273,256, 3]) # explicit size needed for TPU
    return image
    
def prepare_target(target):    
    target = tf.cast(target, tf.float32)            
    target = tf.reshape(target, [1])         
    return target

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image" : tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "target": tf.io.FixedLenFeature([], tf.float32),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    target = prepare_target(example['target'])
    return image, target # returns a dataset of (image, label) pairs

In [ ]:
TRAINING_FILENAMES = tf.io.gfile.glob('train*.tfrec')
TRAINING_FILENAMES

In [ ]:
def load_dataset(fileids, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(fileids) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(20, seed=SEED)
    dataset = dataset.batch(6)
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [ ]:
def display_batch(batch, size=2):
    imgs, tars = batch
    for img_idx in range(size):
        plt.figure(figsize=(4*2, 12*2))
        for idx in range(3):
            plt.subplot(size, 3, idx+1)
            plt.title(f'Target:{tars[img_idx].numpy()[0]}')
            plt.imshow(imgs[img_idx,:, :, idx])
            plt.text(5, 10, str(idx), bbox={'facecolor': 'white'})
            plt.xticks([])
            plt.yticks([])
        plt.tight_layout()
        plt.show() 

In [ ]:
if SANITY_CHECK:
    training_dataset = get_training_dataset()
    training_dataset = training_dataset.unbatch().batch(20)
    train_batch = next(iter(training_dataset))
    display_batch(train_batch, 2);

In [ ]:
if SANITY_CHECK:
    print(np.sum(sgnl - train_batch[0][3].numpy()))

In [ ]:
#image stats
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', img_std)